# Compilation
Compile destination C file.  
Input: name of file without extension format
Output: finished / failed

In [63]:
import os
import sys
def compileCode(filename):
    os.system('gcc source_princetonlibgloballib/'+filename+'.c -lm -o '+filename)
    after_name = filename+".exe"
    print("Blackbox Model Name: ",filename)
    if(os.path.exists(after_name)):
        print("Compilation finished")
    else:
        print("Compilation failed")

# Read boundaries, starting points and number of variables
Read parameters from files in problemdata/ directory  
input: data file name without extension format
output: number of variables, lower bound, upper bound and starting point

In [64]:
# read data file to get values of number of vars, boundaries nad starting points

def read_datafile(filename):
    numOfVar = 0
    lowBound = []
    upBound = []
    startPoint = []
    
    infile = open("problemdata/"+filename+".problem.data",'r')
    lines = infile.readlines()
    # The first line
    for num in lines[0].split():
        numOfVar = int(num.strip())
    # The second line
    for i in lines[1].split():
        lowBound.append(float(i.strip()))
    # The third line
    for j in lines[2].split():
        upBound.append(float(j.strip()))
    # The fourth line
    for k in lines[3].split():
        startPoint.append(float(k.strip()))
    infile.close()
    print("Number of Variables: ",numOfVar)
    print("Lower Boundary: ",lowerBound)
    print("Upper Boundary: ",upperBound)
    print("Starting point is: ",startPoint)
    return numOfVar,lowBound,upBound,startPoint

# Sampling
## Hammersley sequence
Generate input values according to Hammersley sequence which is a classical low discrepency sequence making points spread evenly in multi-dimentional space  
@Parameters:  
Input, integer I, the index of the element of the sequence.  0 <= I.    
Input, integer M, the spatial dimension.  1 <= M <= 100.  
Input, integer N, the "base" for the first component.   1 <= N.  
Output, real R(M), the element of the sequence with index I.

In [30]:
import numpy as np
def hammersley (i,m=1,n=16):
    i = int ( i )
    t = np.ones ( m - 1 )
    t = i * t
#
#  Carry out the computation.
#
    prime_inv = np.zeros ( m - 1 )
    for j in range ( 0, m - 1 ):
        prime_inv[j] = 1.0 / float ( prime ( j ) )
    r = np.zeros ( m )
    r[0] = float ( i % ( n + 1 ) ) / float ( n )
    while ( 0 < np.sum ( t ) ):
        for j in range ( 0, m - 1 ):
            d = ( t[j] % prime ( j ) )
            r[j+1] = r[j+1] + float ( d ) * prime_inv[j]
            prime_inv[j] = prime_inv[j] / prime ( j )
            t[j] = ( t[j] // prime ( j ) )
    return r

def hammersley_seq_format(lBound,uBound,points=16):
    ratio = (uBound - lBound) / 1.0
    seq = []
    for i in range(1,points+1):
        val = hammersley(i)[0]*ratio + lBound
        seq.append(val)
    return seq

# Generate output values
@Parameters  
filename: name of black box model without extension  
variables: values of variables, in list format  
sequence: low discrepency sequence above, in list format  
index: index of the unfixed variable

In [119]:
def write_input(filename,input_values):
    infile = open(filename, 'w')
    for val in input_values:
        infile.write(str(val)+'\n')
    infile.close()
    
def read_output(filename,output_values):
    readfile = open(filename, 'r')
    for line in readfile.readlines():
        output_values.append(float(line.strip()))
    readfile.close()
    
def generate_bbox_values(filename,variables,sequence,index):
    input_filename = "input.in"
    output_filename = "output.out"
    output_values = []
    input_values = []
    for val in sequence:
        input_copy = variables[:]
        input_copy[index] = val
        input_values.append(input_copy)
        write_input(input_filename,input_copy)
        os.system('.\\'+filename)
        read_output(output_filename,output_values)
    return output_values,input_values

def check_bbox_optimal(filename,coordinate):
    input_filename = "input.in"
    output_filename = "output.out"
    output_values = []
    write_input(input_filename,coordinate)
    os.system('.\\'+filename)
    read_output(output_filename,output_values)
    return output_values[0]

# Regression
Use alamopy package to get the numerical expression  
@Parameters  
input_values:  
output_values:  
lowerBound:  
upperBound:  

In [83]:
import alamopy
from sklearn.model_selection import train_test_split
def call_alamopy(input_values,output_values,lowerBound,upperBound):
    X_train,X_test,y_train,y_test=train_test_split(input_values,output_values,test_size=0.25)
    print(lowerBound,upperBound)
    alamo_result = alamopy.alamo(X_train,y_train,xval=X_test,zval=y_test,xmin=lowerBound,xmax=upperBound,monomialpower=(1,2),multi2power=(1,2), showalm=True)
    print("===============================================================")
    print("ALAMO results")
    print("===============================================================")

    print("#Model expression: ",alamo_result['model'])
    print("#Rhe sum of squared residuals: ",alamo_result['ssr'])
    print("#R squared: ",alamo_result['R2'])
    print("#Root Mean Square Error: ",alamo_result['rmse'])
    print("---------------------------------------------------------------")
    labels = alamo_result['xlabels']
    expr = alamo_result['f(model)']
    return labels,expr

# Optimization
Call baron by pyomo to get optimal solution

In [126]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

def boundary_dic(labels,startPoint,index,lb,ub):
    lowerBound = {}
    upperBound = {}
    for (label,val) in zip(labels,startPoint):
        lowerBound[label] = val
    for (label,val) in zip(labels,startPoint):
        upperBound[label] = val
    lowerBound[labels[index]] = lb[index]
    upperBound[labels[index]] = ub[index]
    return lowerBound,upperBound

def call_baron(labels,expr,lowerBound,upperBound,startPoint,index):
    model = ConcreteModel(name='cycle')
    lBound_dic,uBound_dic = boundary_dic(labels,startPoint,index,lowerBound,upperBound)
    def fb(model,i):
        return (lBound_dic[i],uBound_dic[i])
    model.A = Set(initialize=labels)
    model.x = Var(model.A,within=Reals,bounds=fb)
    
    def objrule(model):
        var_lst = []
        for var_name in model.x:
            var_lst.append(model.x[var_name])
        return expr(var_lst)
    model.obj = Objective(rule=objrule,sense=minimize)
    opt = SolverFactory('baron')
    solution = opt.solve(model)
    solution.write()
    model.pprint()
    model.display()
    
    obj_point = []
    for var_name in labels:
        obj_point.append(value(model.x[var_name]))
    obj_value = value(model.obj)
    return obj_point,obj_value

# updating step

In [101]:
def variable_range(lowerBound,upperBound,startPoint,index,ratio=1):
    step = 1.0
    new_lowerBound = startPoint[:]
    new_upperBound = startPoint[:]
    new_lowerBound[index] = startPoint[index] - step
    new_upperBound[index] = startPoint[index] + step
    return new_lowerBound,new_upperBound

# main function
Integration of functions above

In [127]:
blackbox_name = "branin"
compileCode(blackbox_name)
numOfVar,lowerBound,upperBound,startPoint = read_datafile(blackbox_name)
for indexOfVar in range(len(startPoint)):
    lb,ub = variable_range(lowerBound,upperBound,startPoint,indexOfVar)
    var_seq = hammersley_seq_format(lb[indexOfVar],ub[indexOfVar])
    ydata,xdata = generate_bbox_values(blackbox_name,startPoint,var_seq,indexOfVar)
    labels,expr = call_alamopy(xdata,ydata,lowerBound,upperBound)
    optimal_point,optimal_val = call_baron(labels,expr,lb,ub,startPoint,indexOfVar)
    box_val = check_bbox_optimal(blackbox_name,optimal_point)
    print(optimal_val,box_val)
    break

Blackbox Model Name:  branin
Compilation finished
Number of Variables:  2
Lower Boundary:  [-5.0, 0.0]
Upper Boundary:  [10.0, 15.0]
Starting point is:  [2.5, 7.5]
[-5.0, 0.0] [10.0, 15.0]
ALAMO results
#Model expression:    z1 =  - 13.918761705215544566272 * x1 + 0.61544456433306915510428E-001 * (x1*x2)^2 + 37.277925581735232185565
#Rhe sum of squared residuals:   0.425E-01
#R squared:   0.999
#Root Mean Square Error:   0.595E-01
---------------------------------------------------------------
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: problem
  Lower bound: -59.32949989
  Upper bound: -59.32356694
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 3
  Sense: unk